# Pandas Student Notebook — Foundations Practice (4)  
## Dataset: Kaggle “Brazilian E-Commerce Public Dataset by Olist” (subset)

### Goal of this notebook
This notebook focuses on **relational thinking in Pandas**: multiple tables, grain alignment, aggregation before joins, and metric correctness.
You will practice combining tables, validating assumptions, and avoiding common analytical traps.

Important mindset:
Every mistake in this notebook will still produce *numbers* — your job is to make sure they produce **meaning**.

Write your code in the empty code cells.


## 0. Setup + data model awareness

Load the following CSV files:
- `orders.csv`
- `order_items.csv`
- `customers.csv`
- `payments.csv`

Display:
- shape and `info()` for each table

Write as a comment:
- What is the grain of each table?
- Which columns are keys vs attributes?



orders: (99441, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB

order_items: (112650, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0  

## 1. Orders timeline sanity checks

1) Convert all order timestamp columns to datetime.  
2) Create `purchase_year` and `purchase_month`.

Validate:
- `order_delivered_customer_date >= order_purchase_timestamp` where both exist and count violations
- Verify that all datetime columns are timezone-naive (i.e. they do not contain any timezone information such as UTC offsets).


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
 8   purchase_year                  99441 non-null  int32         
 9   purchase_month                 99441 non-null  object        
dtypes: datetime64[ns](5), int32(1), object(4)
memory usage: 7.2+ MB


Delivery-before-purchase violations: 0


Any timezone present? False


## 2. Row explosion awareness

Compute:
- number of orders
- number of order_items
- average number of items per order

Then show what happens if you:
- merge `orders` with `order_items`
- and call `len()` on the result

Write as a comment:
- Why `len(orders_items_merged)` is meaningless without context


number of orders: 99441
number of order_items rows: 112650
avg items per order: 1.1417306873695092
len(orders_items_merged): 113425


## 3. Revenue per order (correctly)

Compute total order revenue as:
`sum(price + freight_value)` per order

Constraints:
- Must originate from `order_items`
- Exactly one row per order
- No loops
- No double counting

Write as a comment:
- One tempting but wrong approach, and why it is wrong


,order_id,order_total_value
0,00010242fe8c5a6d1ba2dd792cb16214,72.19
1,00018f77f2f0320c557190d7a144bdd3,259.83
2,000229ec398224ef6ca0657da4fc703e,216.87
3,00024acbcdf0a6daa1e931b038114c75,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04


## 4. Payment aggregation before logic

Orders can have multiple payment rows.

1) Aggregate `payments` to order-level:
- total payment value
- number of payment rows

2) Merge into `orders`.

Write as a comment:
- Why payments must be aggregated before merging with orders


,order_id,payment_total_value,n_payment_rows
0,e481f51cbdc54678b7cc49136f2d6af7,38.71,3.0
1,53cdb2fc8bc7dce0b6741e2150273451,141.46,1.0
2,47770eb9100c2d0c44946d9cf07ec65d,179.12,1.0
3,949d5b44dbf5de918fe9c16f97b45f8a,72.20,1.0
4,ad21c59c0840e6cb83a9ceb5573f8159,28.62,1.0


## 5. Data quality rule across tables

Create boolean `order_data_suspicious` if:
- order is marked as delivered
- AND no payment exists
- OR total payment value == 0

Show:
- count of suspicious orders
- sample rows



suspicious orders: 1


,order_id,order_status,payment_total_value,n_payment_rows
30710,bfbd0f9bdef84302105ad712db648a6c,delivered,0.0,NaN


## 6. Groupby + transform: relative order size

For each customer:
- compute average order value
- attach it back to each order
- compute `order_vs_customer_avg`

Show sample rows.




,order_id,customer_unique_id,order_total_value,customer_avg_order_value,order_vs_customer_avg
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,38.71,41.41,0.934798
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,141.46,141.46,1.000000
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,179.12,179.12,1.000000
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,72.20,72.20,1.000000
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,28.62,28.62,1.000000
5,a4591c265e18cb1dcee52889e2d8acc3,80bb27c7c16e8f973207a5086ab329e2,175.26,175.26,1.000000
6,136cce7faa42fdb2cefd53fdc79a6098,36edbb3fb164b1f16485364b6fb04c73,65.95,65.95,1.000000
7,6514b8ad8028c9f2cc2374ded245783f,932afa1e708222e5821dac9cd5db4cae,75.16,75.16,1.000000
8,76c6e866289321a7c93b82b54852dc33,39382392765b6dc74812866ee5ee92a7,35.95,35.95,1.000000
9,e69bfb5eb88e0ed6a785585b27e16dbf,299905e3934e9e181bfb2e164dd4b4f8,169.76,169.76,1.000000


## 7. Customer cohorts (monthly)

Define:
- cohort = month of first purchase

Create a cohort table:
- rows = cohort month
- columns = months since first purchase
- values = number of active customers

Constraints:
- Pandas only
- No black-box shortcuts

Write as a comment:
- What question this answers
- What it does NOT answer


months_since_first,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20
cohort_month,,,,,,,,,,,,,,,,,,,,
2016-09,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-10,321,0,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,2,2
2016-12,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01,764,3,2,1,3,1,4,1,1,0,3,1,6,3,1,1,2,3,1,0
2017-02,1752,4,5,2,7,2,4,3,3,4,2,5,3,3,2,1,1,4,0,0


## 8. Seller concentration per state

For each seller:
- compute total revenue per year
- compute total revenue per state-year
- compute seller share of state revenue
- rank sellers per state-year
- keep top 5 per state-year

Constraints:
- No Python loops
- No merges after aggregation

Write as a comment:
- Why window logic is required here


,customer_state,purchase_year,seller_id,seller_state_year_revenue,state_year_revenue,seller_share_of_state,rank_in_state_year
15,AC,2017,53243585a1d6dc2643021fd1853d8905,1251.70,12986.60,0.096384,1
28,AC,2017,7e93a43ef30c4f03f38b393420bc753a,995.18,12986.60,0.076631,2
7,AC,2017,25c5c91f63607446a97b143d2d535d31,861.26,12986.60,0.066319,3
41,AC,2017,cca3071e3e9bb7d12640c9fbe2301306,723.14,12986.60,0.055684,4
23,AC,2017,76d5af76d0271110f9af36c92573f765,618.60,12986.60,0.047634,5
67,AC,2018,c4d51195486dc781531876a7d00453d8,905.93,6683.10,0.135555,1
53,AC,2018,527801b552d0077ffd170872eb49683b,757.51,6683.10,0.113347,2
56,AC,2018,5a8e7d5003a1f221f9e1d6e411de7c23,646.44,6683.10,0.096728,3
47,AC,2018,04308b1ee57b6625f47df1d56f00eedf,591.88,6683.10,0.088564,4
57,AC,2018,626ab1cd96932367f0eeec2e47046c12,548.93,6683.10,0.082137,5


## 9. Rolling customer behavior (order-based)

For each customer:
- sort by purchase date
- compute rolling 3-order average order value
- use only previous orders (no leakage)
- preserve original row order

Write as a comment:
- Why this is order-based rolling, not time-based


,order_id,customer_unique_id,order_total_value,rolling_3_order_avg
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,38.71,44.11
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,141.46,0.00
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,179.12,0.00
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,72.20,0.00
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,28.62,0.00
5,a4591c265e18cb1dcee52889e2d8acc3,80bb27c7c16e8f973207a5086ab329e2,175.26,0.00
6,136cce7faa42fdb2cefd53fdc79a6098,36edbb3fb164b1f16485364b6fb04c73,65.95,0.00
7,6514b8ad8028c9f2cc2374ded245783f,932afa1e708222e5821dac9cd5db4cae,75.16,0.00
8,76c6e866289321a7c93b82b54852dc33,39382392765b6dc74812866ee5ee92a7,35.95,0.00
9,e69bfb5eb88e0ed6a785585b27e16dbf,299905e3934e9e181bfb2e164dd4b4f8,169.76,0.00


## 10. Capstone: clean analytical table

Create `analysis_df` with:
- order_id
- customer_unique_id
- purchase_year
- order_total_value
- order_vs_customer_avg
- rolling_3_order_avg
- order_data_suspicious (as int)

Requirements:
- one row per order
- no missing values in engineered metrics
- show `head()` and `isna().sum()`



,order_id,customer_unique_id,purchase_year,order_total_value,order_vs_customer_avg,rolling_3_order_avg,order_data_suspicious
0,e481f51cbdc54678b7cc49136f2d6af7,7c396fd4830fd04220f754e42b4e5bff,2017,38.71,0.934798,44.11,0
1,53cdb2fc8bc7dce0b6741e2150273451,af07308b275d755c9edb36a90c618231,2018,141.46,1.000000,0.00,0
2,47770eb9100c2d0c44946d9cf07ec65d,3a653a41f6f9fc3d2a113cf8398680e8,2018,179.12,1.000000,0.00,0
3,949d5b44dbf5de918fe9c16f97b45f8a,7c142cf63193a1473d2e66489a9ae977,2017,72.20,1.000000,0.00,0
4,ad21c59c0840e6cb83a9ceb5573f8159,72632f0f9dd73dfee390c9b22eb56dd6,2018,28.62,1.000000,0.00,0


order_id                 0
customer_unique_id       0
purchase_year            0
order_total_value        0
order_vs_customer_avg    0
rolling_3_order_avg      0
order_data_suspicious    0
dtype: int64